# 修了課題③ 仮想通貨

**データセット**：仮想通貨の週単位のデータ  
※１週間のうち６日間の取引価格の終値を学習データとして利用して、最後の１日の取引価格の終値を予測する。これは、１週間の取引価格の間に依存関係があり、正しく傾向を学習すれば、６日間の取引価格から残りの１日間の価格が予測できるはずという仮説に基づいたものとなる。

**合格基準**：RMSE 50未満
  

※補足 ( 仮想通貨に関して)  
仮想通貨は、株式と同じように個別銘柄（ビットコインやイーサリアムなど）が存在し、日々仮想通貨市場にて取引が行われている。仮想通貨市場は為替市場などと同じように取引されており、例えば代表的な仮想通貨であるビットコインなどであれば、１ビットコイン＝〇〇ドルのような形で日々刻々と価格が変動している。

# データのダウンロード

In [1]:
# 学習データのダウンロード
!wget 'https://drive.google.com/uc?export=download&id=1kUfPb8qikA8rdQ26iVUxpod2Qjw3ct_O' -O crypto_train.csv

# テストデータのダウンロード
!wget 'https://drive.google.com/uc?export=download&id=1VhzCcjNSDxGRG86Za653zHHpjVCdSPD3' -O crypto_test_x.csv

--2025-01-27 04:22:37--  https://drive.google.com/uc?export=download&id=1kUfPb8qikA8rdQ26iVUxpod2Qjw3ct_O
正在解析主机 drive.google.com (drive.google.com)... 142.251.222.14
正在连接 drive.google.com (drive.google.com)|142.251.222.14|:443... 已连接。
已发出 HTTP 请求，正在等待回应... 303 See Other
位置：https://drive.usercontent.google.com/download?id=1kUfPb8qikA8rdQ26iVUxpod2Qjw3ct_O&export=download [跟随至新的 URL]
--2025-01-27 04:22:38--  https://drive.usercontent.google.com/download?id=1kUfPb8qikA8rdQ26iVUxpod2Qjw3ct_O&export=download
正在解析主机 drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.42.193
正在连接 drive.usercontent.google.com (drive.usercontent.google.com)|142.251.42.193|:443... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度：24244 (24K) [application/octet-stream]
正在保存至: “crypto_train.csv”

crypto_train.csv    100%[===================>]  23.68K  --.-KB/s  用时 0.05s     

2025-01-27 04:22:41 (511 KB/s) - 已保存 “crypto_train.csv” [24244/24244])

--2025-01-27 04:22:41--  https://drive.google.com/uc?export=d

# データの確認

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [37]:
# 学習データの確認
train = pd.read_csv('crypto_train.csv', index_col=0)
train

,Mon,Tue,Wed,Thu,Fri,Sat,Sun
0,144.539993,139.000000,116.989998,105.209999,97.750000,112.500000,115.910004
1,112.300003,111.500000,113.566002,112.669998,117.199997,115.242996,115.000000
2,117.980003,111.500000,114.220001,118.760002,123.014999,123.498001,121.989998
3,122.000000,122.879997,123.889000,126.699997,133.199997,131.979996,133.479996
4,129.744995,129.000000,132.300003,128.798996,129.000000,129.300003,122.292000
...,...,...,...,...,...,...,...
186,739.247986,751.346985,744.593994,740.289001,741.648987,735.382019,732.034973
187,735.812988,735.604004,745.690979,756.773987,777.943970,771.155029,773.872009
188,758.700012,764.223999,768.132019,770.809998,772.794006,774.650024,769.731018
189,780.086975,780.556030,781.481018,778.088013,784.906982,790.828979,790.530029


In [4]:
# テストデータの確認
test = pd.read_csv('crypto_test_x.csv', index_col=0)
test

,Mon,Tue,Wed,Thu,Fri,Sat
0,1021.750000,1043.839966,1154.729980,1013.380005,902.200989,908.585022
1,902.828003,907.679016,777.757019,804.833984,823.984009,818.411987
2,831.533997,907.937988,886.617981,899.072998,895.026001,921.789001
3,921.012024,892.687012,901.541992,917.585999,919.750000,921.590027
4,920.382019,970.403015,989.023010,1011.799988,1029.910034,1042.900024
5,1038.150024,1061.349976,1063.069946,994.382996,988.674011,1004.450012
6,990.642029,1004.549988,1007.479980,1027.439941,1046.209961,1054.420044
7,1079.979980,1115.300049,1117.439941,1166.719971,1173.680054,1143.839966
8,1179.969971,1179.969971,1222.500000,1251.010010,1274.989990,1255.150024
9,1272.829956,1223.540039,1150.000000,1188.489990,1116.719971,1175.829956


# モデルの作成

In [41]:
# モデルを作成し、テストデータに対して予測結果を出力して下さい。
import torch.nn as nn
import torch

class LSTMModel(nn.Module):
    """LSTM モデル"""
    def __init__(self, input_size=1, hidden_size_1=64, hidden_size_2=32, output_size=1):
        super(LSTMModel, self).__init__()
        self.hidden_size_1 = hidden_size_1
        self.hidden_size_2 = hidden_size_2
        
        self.lstm1 = nn.LSTM(input_size, hidden_size_1, batch_first=True)
        self.dropout1 = nn.Dropout(0.2)
        self.lstm2 = nn.LSTM(hidden_size_1, hidden_size_2, batch_first=True)
        self.dropout2 = nn.Dropout(0.2)
        self.fc = nn.Linear(hidden_size_2, output_size)
        
        self.criterion = nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)
    
    def forward(self, x):
        # First LSTM layer
        out, _ = self.lstm1(x)
        out = self.dropout1(out)
        
        # Second LSTM layer
        out, _ = self.lstm2(out)
        out = self.dropout2(out[:, -1, :])  # Get last output
        
        # Output layer
        out = self.fc(out)
        return out

In [44]:
x = train.drop('Sun', axis=1).values
y = train['Sun'].values

from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((152, 6), (39, 6), (152,), (39,))

# 提出形式

In [ ]:
# 提出形式の確認
pred = pd.DataFrame(np.random.rand(16, 1), columns=['Sun'])
# 形状が（16, 1）になっていることを確認して下さい。
print(pred.shape)
pred

In [ ]:
# csv形式での提出をお願いします。
pred.to_csv('crypto_pred.csv')